In [2813]:
import psycopg2
from flask import Flask, session, request, redirect
from markupsafe import escape

app = Flask(__name__)
app.config['SECRET_KEY'] = 'shh_dont_tell'

In [ ]:
#ESTABLISHES CONNECTION FUNCTION- to call
def establish_connection():
    return psycopg2.connect(
    host = "localhost",                 #use own user
    database= "Real Estate System",
    user = "postgres",
    password = "-----",            #use own password
    port = 5432)

In [2793]:
# To Start Website Please Run ALL and then click link 
# provided by __main__ at the end!

In [2794]:
#FUNCTIONS

#Get role

def get_role(User_id, cursor):
    cursor.execute("SELECT * FROM User_renter WHERE userid = %s;", (User_id,))
    if cursor.fetchone() is not None:
        return 'renter'
    else:
        cursor.execute("SELECT * FROM User_agent WHERE userid = %s;", (User_id,))
        if cursor.fetchone() is not None:
            return 'agent'
        else:
            return "Invalid"
    

In [2795]:
@app.route("/")
def home():
    user_id = session.get('UserID')
    if user_id:
        return redirect(f'/dashboard/{user_id}')
    return redirect('/Login')

In [2796]:
@app.route('/dashboard/<user_id>')
def dashboard(user_id):

    return f'''<!DOCTYPE html>
    <html>
    <head>
            <title> Dashboard! </title>
    </head>
    <body>
        <h1 style= "text-align:center"> Dashboard for user {escape(user_id)}! </h1>

    <form method = "GET">
                <a href = "/CC" style = "display: inline-block; padding:10px 15px; background-color:#d4cfd6; color = black; border-radius: 5px; target="_self"> Manage Credit Cards </a>
                <a href = "/Properties"style = "display: inline-block; padding:10px 15px; background-color:#d4cfd6; color = black; border-radius: 5px; target="_self">Search Properties</a>
                <a href = "/Bookings" style = "display: inline-block; padding:10px 15px; background-color:#d4cfd6; color = black; border-radius: 5px; target="_self"> Manage Bookings</a>
                <a href = "/BookProp" style = "display: inline-block; padding:10px 15px; background-color:#d4cfd6; color = black; border-radius: 5px; target="_self"> Book Properties</a>
                <a href = "/Login" style = "display: inline-block; padding:10px 15px; background-color:#d4cfd6; color = black; border-radius: 5px; target="_self"> Login </a>
                <a href = "/register" style = "display: inline-block; padding:10px 15px; background-color:#d4cfd6; color = black; border-radius: 5px; target="_self"> Register</a>
                <a href = "/Address_Update" style = "display: inline-block; padding:10px 15px; background-color:#d4cfd6; color = black; border-radius: 5px; target="_self">Update Address</a>

     </form>
    </body>
    </html>'''


In [2797]:
@app.route("/Login", methods=['GET','POST'])
def Login():
    if request.method == 'POST':
        email_address = request.form['email_address']
        try:
            conn = establish_connection()

            cursor = conn.cursor()
            cursor.execute('SELECT UserID FROM "user" WHERE email_address = %s', (email_address,))
            result = cursor.fetchone()

            if result:
                session['UserID'] = result[0]
                return redirect('/')
            else:
                return redirect('/register')
        except Exception as e:
            return f"Login error: {e}"
        finally:
            if conn is not None:
                conn.close()

    return '''<!DOCTYPE html>
    <html>
    <body>

    <h2>Real Estate Login</h2>

    <form action="/Login" method = "POST">
    <label for="Email">Email Address:</label><br>
    <input type="text" id="Email" name="email_address"><br>
    <input type = "submit" value = "Login">
    </form>
    </body>
    </html>'''


In [2798]:
@app.route('/register', methods=['GET','POST'])
def register():
    conn = establish_connection()
    cursor = conn.cursor()

    if request.method == 'POST':
        try:
            email_address = request.form['email_address']
            first_name = request.form["first_name"]
            mid_name =request.form["mid_name"]
            last_name = request.form["last_name"]
            role = request.form['role'].lower()

            cursor.execute('INSERT INTO "user" (email_address,first_name,mid_name,last_name) VALUES (%s,%s,%s,%s) RETURNING UserID;',(email_address,first_name,mid_name,last_name))
            user_id = cursor.fetchone()[0]
            
            if role =='renter':
                moveindate = request.form['moveindate']
                location= request.form['location']
                budget = request.form['budget']

                cursor.execute("INSERT INTO User_renter (UserID, moveindate, location, budget) VALUES (%s,%s,%s,%s);",(user_id,moveindate,location, budget))          

            elif role =='agent':
                job_title = request.form['job_title']
                agency = request.form['agency']
                phone_num = request.form['phone_num']

                cursor.execute("INSERT INTO User_agent (UserID, job_title,agency, phone_num) VALUES (%s,%s,%s,%s);",(user_id,job_title,agency,phone_num))
            conn.commit()
            session['UserID'] = user_id
            return redirect('/Address')

        except Exception as e:
            return f"Registration error: {e}"
        finally:
            if conn is not None:
                conn.close()
        
    return '''
<!DOCTYPE html>
<html>
<body>

<h2>Real Estate Registration</h2>

<form method = "POST">
  <label for="Email">Email Address:</label><br>
  <input type="text" id="Email" name="email_address"><br>

  <label>First Name:</label><br>
  <input type="text" name="first_name"><br>

   <label> Middle Name:</label><br>
  <input type="text" name="mid_name"><br>

   <label>Last Name:</label><br>
  <input type="text" name="last_name"><br>

   <label>Role (renter or agent):</label><br>
  <input type="text" name="role"><br><br>

  <strong>Renter Info:</strong><br>

   <label>Move-In Date:</label><br>
  <input type="text" name="moveindate"><br>

   <label>Location:</label><br>
  <input type="text" name="location"><br>

   <label>Budget:</label><br>
  <input type="text" name="budget"><br><br>

  <strong>Agent Info:</strong><br>

   <label>Job Title:</label><br>
  <input type="text" name="job_title"><br>

   <label>Agency:</label><br>
  <input type="text" name="agency"><br>

   <label>Phone Number:</label><br>
  <input type="text" name="phone_num"><br><br>

  <input type="submit" value="Register">
</form> 
</body>
</html>'''

            

In [2799]:
@app.route("/Address", methods=['GET','POST'])
def Address():
   conn= establish_connection()
   cursor = conn.cursor()
   user_id = session.get('UserID')

   if not user_id:
      return redirect('/Login')
   
   
   try:
         if request.method == 'POST':

         
            
            number = request.form['number']
            street = request.form['street'] 
            apt = request.form['apt']
            city = request.form['city']
            state=request.form['state']
            zip=request.form['zip']
            country=request.form['country']

            cursor.execute('INSERT INTO Address VALUES (%s,%s,%s,%s,%s,%s,%s,%s);',(user_id,number,street,apt,city,state,zip,country))  
            conn.commit()

            return redirect('/')
               
   except Exception as e:
         return f"CC error: {e}"
   finally:
         if conn is not None:
            conn.close()

   return '''<!DOCTYPE html>
         <html>
         <body>

         <h2>Real Address Input:</h2>

         <form method = "POST">


         <label>House Number:</label><br>
         <input type="text" name="number"><br>

            <label> Street:</label><br>
         <input type="text" name="street"><br>

            <label>APT number:</label><br>
         <input type="text" name="apt"><br>

            <label>City:</label><br>
         <input type="text" name="city"><br><br>


            <label>State:</label><br>
         <input type="text" name="state"><br>

            <label>ZIP Code:</label><br>
         <input type="text" name="zip"><br>

            <label>Country:</label><br>
         <input type="text" name="country"><br><br>


         <input type="submit" value="Submit">
         </form> 
         </body>
         </html>'''

In [2800]:
@app.route("/Address_Update", methods=['GET','POST'])
def Address_Update():
   conn= establish_connection()
   cursor = conn.cursor()
   user_id = session.get('UserID')

   if not user_id:
      return redirect('/Login')
   
   
   try:
         if request.method == 'POST':

         

            field= request.form['field']
            new_value= request.form['new_value']

            
            cursor.execute("SELECT * FROM Address WHERE UserID = %s::SMALLINT;", (user_id,))
            if not cursor.fetchone():
               return redirect('/Address')
         
            fields_available = ['number','street','apt','city','state','zip','country']
            if field in fields_available:
               query = f"UPDATE Address SET {field} = %s WHERE UserID = %s;"
               cursor.execute(query,(new_value,user_id))
               conn.commit()
               return redirect('/')
               
            
   except Exception as e:
         return f"CC error: {e}"
   finally:
         if conn is not None:
            conn.close()

   return '''   <!DOCTYPE html>
      <html lang="en">
      <body>
         <h1> Update Address </h1>
      </body>

      <form method = "POST"> 

         
         <label for="field">Field to change:</label>
         <select name = "field">
            <option value="num"> House Number</option>
            <option value="apt"> APT</option> 
            <option value="street"> Street</option> 
            <option value="city"> City</option>
            <option value="state"> State</option> 
            <option value="zip"> ZIP </option> 
            <option value="country"> Country</option> 
         </select>

         New Value:
      <input type="text" name="new_value"><br>


      <input type="submit" value="Update">
      </form> 
      </body>
      </html>
      '''


In [2801]:
@app.route("/CC")
def CC():
    conn = establish_connection()
    cursor = conn.cursor()

    user_id = session.get('UserID')
    
    if not user_id:
        return redirect('/Login')
    
    cursor.execute("SELECT * FROM CC WHERE UserID = %s;",(user_id,))
    cards = cursor.fetchall()
    cursor.close()
    conn.close()

    rows = ""
    for card in cards:
        row = "<tr>" + "".join(f"<td>{value}</td>" for value in card) + "</tr>\n"
        rows+= row



    return f'''
    <!DOCTYPE html>
<html lang="en">
 
<head>
    <meta charset="UTF-8">
    <title> Credit Card System</title>

    <style>
        table {{margin: 0 auto, font-size: medium; border: 1px solid black;}}

        h1 {{text-align: center; color =black; font-size: large; font-family: 'Calibri' }}
        td {{background-color:#E4F5D4; border: 1px solid black;}}
        th,
        td{{font-weight: bold; border: 1px solid black; padding: 10px; text-align: center;}}

        td {{font-weight: lighter;}}
    </style>
</head>
<body>
    <h1> Your Credit Cards<h1>
    <form method = "GET">
                <a href = "/CC_Add" style = "display: inline-block; padding:10px 15px; background-color:#d4cfd6; color = black; border-radius: 5px; target="_self"> Add A Credit Card </a>
                <a href = "/CC_Delete" style = "display: inline-block; padding:10px 15px; background-color:#d4cfd6; color = black; border-radius: 5px; target="_self"> Delete A Credit Card </a>
                <a href = "/CC_Update" style = "display: inline-block; padding:10px 15px; background-color:#d4cfd6; color = black; border-radius: 5px; target="_self"> Update A Credit Card </a>
                <a href = "/dashboard/{user_id}" style = "display: inline-block; padding:10px 15px; background-color:#d4cfd6; color = black; border-radius: 5px; target="_self"> Back to Dashboard </a>
    <section>
    <table>
        <tr>
            <th>CardID</th>
            <th>CC Number</th>
            <th>UserID</th>
            <th>House Number</th>
            <th>House APT</th>
            <th>Street</th>
            <th>City</th>
            <th>State</th>
            <th>ZIP</th>
            <th>Country</th>
            <th>Expiration Date</th>
            <th>Cardholder Name</th>
        </tr>
        {rows}
        </table>
        </section>
    </body>
    </html>

'''


    

In [2802]:
@app.route("/CC_Add", methods=['GET','POST'])
def add_CC():
   conn= establish_connection()
   cursor = conn.cursor()
   user_id = session.get('UserID')
   print('UserID:',user_id)

   if not user_id:
            return redirect('/Login')
   
   try:
      if request.method == 'POST':  
         
         role = get_role(user_id,cursor)

         if role != 'renter':
            print("Access denied!")
            return redirect('/')
   
         
         CC_num = request.form['CC_num']
         bill_num=request.form['bill_num']
         bill_apt=request.form['bill_apt']
         bill_street=request.form['bill_street']
         bill_city=request.form['bill_city']
         bill_state=request.form['bill_state']
         bill_zip=request.form['bill_zip']
         bill_country=request.form['bill_country']
         Expiration_date=request.form['Expiration_date']
         Cardholder_Name=request.form['Cardholder_Name']

         cursor.execute("CALL add_CC2(%s,%s::SMALLINT,%s,%s,%s,%s,%s,%s,%s,%s,%s);",(CC_num,user_id,bill_num,bill_apt,bill_street,bill_city,bill_state,bill_zip,
            bill_country,Expiration_date,Cardholder_Name))
         conn.commit() 
         return redirect('/CC')
      
   except Exception as e:
      return f"CC error: {e}"
   finally:
      if conn is not None:
         conn.close()

   return'''   <!DOCTYPE html>
<html lang="en">
<head>
    <h1> Add a Credit Card </h1>
</head>

<form method = "POST">
   
  CC Number:
  <input type="text" name="CC_num"><br>

   Expiration Date:
  <input type="text" name="Expiration_date"><br>

   Cardholder Name:
  <input type="text" name="Cardholder_Name"><br>

   <strong>Card Address:</strong><br>

   House Number:
  <input type="text" name="bill_num"><br>

   APT:
  <input type="text" name="bill_apt"><br>

   Street:
  <input type="text" name="bill_street"><br>

   City:
  <input type="text" name="bill_city"><br>

   State:
  <input type="text" name="bill_state"><br>

   Zip Code:
  <input type="text" name="bill_zip"><br>

   Country:
  <input type="text" name="bill_country"><br>

  <input type="submit" value="Add">
</form> 
</body>
</html>
'''

In [2803]:
@app.route("/CC_Delete", methods=['GET','POST'])
def delete_CC():
   conn= establish_connection()
   cursor = conn.cursor()
   user_id = session.get('UserID')

   if not user_id:
            return redirect('/Login')
   
   try:
      if request.method == 'POST':  
         
         role = get_role(user_id,cursor)
         CC_num=request.form['CC_num']


         if role != 'renter':
            print("Access denied!")
            return redirect('/')
        
         cursor.execute("CALL delete_CC2(%s,%s::SMALLINT);",(CC_num, user_id))  # FUNCTION
         conn.commit()
         return redirect('/CC')
      
   except Exception as e:
      return f"CC error: {e}"
   finally:
      if conn is not None:
         conn.close()
   return '''   <!DOCTYPE html>
<html lang="en">
<body>
    <h1> Delete a Credit Card </h1>
</body>

<form method = "POST"> 

   CC Number:
  <input type="text" name="CC_num"><br>

  <input type="submit" value="Delete">
</form> 
</body>
</html>
'''

In [2804]:
@app.route("/CC_Update", methods=['GET','POST'])
def update_CC():
   conn= establish_connection()
   cursor = conn.cursor()
   user_id = session.get('UserID')

   if not user_id:
      return redirect('/Login')
   
   try:
      if request.method == 'POST':

         
         role = get_role(user_id,cursor)
         if role != 'renter':
            return "Access denied!", 403
       
         CardID = request.form['CardID']
         field= request.form['field']
         new_value= request.form['new_value']

         
         cursor.execute("SELECT * FROM CC WHERE CardID = %s::SMALLINT AND UserID = %s::SMALLINT;", (CardID, user_id))
         if not cursor.fetchone():
            return redirect('/CC')
       
         fields_available = ['CC_num','bill_num','bill_apt','bill_street','bill_city','bill_state','bill_zip','bill_country','Expiration_date','Cardholder_Name']
         if field in fields_available:
            query = f"UPDATE CC SET {field} = %s WHERE CardID = %s AND UserID = %s;"
            print(query)
            print(cursor.closed)
            cursor.execute(query,(new_value, CardID, user_id))
            conn.commit()
            return redirect('/CC')
   

   except Exception as e:
      return f"CC error: {e}"
   finally:
      if conn is not None:
         conn.close()

   return '''   <!DOCTYPE html>
<html lang="en">
<body>
    <h1> Update a Credit Card </h1>
</body>

<form method = "POST"> 

   CCID:
  <input type="text" name="CardID"><br>
   <label for="field">Field to change:</label>
   <select name = "field">
      <option value="CC_num"> CC_num</option>
      <option value="bill_num"> bill_num</option> 
      <option value="bill_apt"> bill_apt</option> 
      <option value="bill_street"> bill_street</option> 
      <option value="bill_city"> bill_city</option>
      <option value="bill_state"> bill_state</option> 
      <option value="bill_zip"> bill_zip</option> 
      <option value="bill_country"> bill_country</option> 
      <option value="Expiration_date"> Expiration_date</option>
      <option value="Cardholder_Name"> Cardholder_Name/option>
   </select>

   New Value:
  <input type="text" name="new_value"><br>


  <input type="submit" value="Update">
</form> 
</body>
</html>
'''



In [2805]:
@app.route("/Properties", methods=['GET','POST'])
def searchProperties():
   conn= establish_connection()
   cursor = conn.cursor()
   user_id = session.get('UserID')
   rows = ""

   if not user_id:
            return redirect('/Login')
   
   try:
      if request.method == 'POST':
         
         CardID = request.form['CardID']
         city =request.form['city']
         state = request.form['state']
         bd_limit = request.form['bd_limit']
         price_range =request.form['price_range']
         property_type =request.form['property_type']
         order_by = request.form['order_by']
         buy_type = request.form['buy_type']
           
      
         
         query = ("""SELECT p.propertyID, p.price, a.number, a.street,a.apt,a.zip,a.country,h.room_num ,p."type", p.description FROM property p JOIN Prop_Adress a ON p.propertyID = a.propertyID
         LEFT JOIN property_house h ON p.propertyID = h.propertyID LEFT JOIN property_apt ap ON p.propertyID = ap.propertyID
         WHERE a.city = %s and a.state=%s """)

         conditions =[]
         values=[city, state]

         if price_range:
            conditions.append("p.price <= %s")
            values.append(price_range)   
         if bd_limit:
            conditions.append("(h.room_num<= %s OR ap.aptroom_num <= %s)")
            values.extend([bd_limit,bd_limit])
         if property_type == "apartment":
            conditions.append("ap.propertyID is NOT NULL")
         if property_type == "house":
            conditions.append("h.propertyID is NOT NULL")
         if property_type == "commercial":
            conditions.append("p.propertyID IN (SELECT propertyID FROM property_commercial)")

         if buy_type.lower() == "rent":
            conditions.append("p.availability_status = 'rent'")
         elif buy_type.lower()== "buy":
            conditions.append("p.availability_status = 'sale'")    

         if conditions:
            query += ' AND ' + ' AND '.join(conditions)

         if order_by == 'price':
            order_clause = ("ORDER BY p.price ASC")
         elif order_by == "bedrooms":
            order_clause = ("ORDER BY h.room_num ASC")
         else:
            order_clause = ""

         final_query = query + order_clause

         cursor.execute(final_query,values)
         results = cursor.fetchall()

         for result in results:
            row = "<tr>" + "".join(f"<td>{value}</td>" for value in result) + "</tr>\n"
            rows+= row

   


      return f'''<!DOCTYPE html>
         <html lang="en">
         <head>
         <meta charset="UTF-8">
         <title> Properties </title>

         <style>
            table {{margin: 0 auto, font-size: medium; border: 1px solid black;}}

            h1 {{text-align: center; color:black; font-size: large; font-family: 'Calibri'; }}
            td {{background-color:#E4F5D4; border: 1px solid black;}}
            th,
            td{{font-weight: bold; border: 1px solid black; padding: 10px; text-align: center;}}

            td {{font-weight: lighter;}}
         </style>
         </head>

         <body>

         <h1> Properties </h1>
            <form method = "POST"> 

            State (Required):
               <input type="text" name="state"><br>
            City (required):
               <input type="text" name="city"><br>
            Bedroom Limit:
               <input type="text" name="bd_limit"><br>
            Price Limit:
               <input type="text" name="price_range"><br>
            Type (apartment,house,or commercial):
               <input type="text" name="property_type"><br>
            Order By (price or bedrooms):
               <input type="text" name="order_by"><br>
            Buy Type:
               <input type="text" name="buy_type"><br>
            Card ID:
               <input type="text" name="CardID"><br>

               <br>
               <input type="submit" value="Search">
               <br>
               </form> 
         <table>
            <tr>
               <th>Property ID </th>
               <th> Price </th>
               <th> Number</th>
               <th> Street </th>
               <th> APT </th>
               <th> ZIP </th>
               <th> Country </th>
               <th> Rooms </th>
               <th> Type </th>
               <th> Description </th>
            </tr>
            {rows if rows else '<tr><td colspan="5">No properties found.</td></tr>'}
         </table>
         <br>
         <a href = "/dashboard/{user_id}" style = "display: inline-block; padding:10px 15px; background-color:#d4cfd6; color = black; border-radius: 5px; target="_self"> Back to Dashboard </a>
         <a href = "/Delete_Properties" style = "display: inline-block; padding:10px 15px; background-color:#d4cfd6; color = black; border-radius: 5px; target="_self"> Delete a Property </a>
         <a href = "/Prop_Update" style = "display: inline-block; padding:10px 15px; background-color:#d4cfd6; color = black; border-radius: 5px; target="_self"> Update a Property </a>

     
      </body>
      </html>  
      '''
   except Exception as e:
      return f"CC error: {e}"
   finally:
      if conn is not None:
         conn.close()

In [2806]:
@app.route("/Delete_Properties", methods=['GET','POST'])
def deleteProperties():
   conn= establish_connection()
   cursor = conn.cursor()
   user_id = session.get('UserID')
   

   if not user_id:
            return redirect('/Login')
   
   try:
      if request.method == 'POST':
         role = get_role(user_id,cursor)
         
         if role != 'agent':
            print("Access denied!")
            return redirect('/')
         
         propertyID_input = request.form['propertyID_input']
        
         cursor.execute("CALL deletePROP(%s,%s);",(propertyID_input,user_id))
         conn.commit()
           

   except Exception as e:
      return f"CC error: {e}"
   finally:
      if conn is not None:
         conn.close()
   return f'''<!DOCTYPE html>
      <html lang="en">
      <body>
         <h1> Delete a Property </h1>
         <br>
         <a href = "/dashboard/{user_id}" style = "display: inline-block; padding:10px 15px; background-color:#d4cfd6; color = black; border-radius: 5px; target="_self"> Back to Dashboard </a>


      <form method = "POST"> 

         Enter Property Number Under your UserID:
      <input type="text" name="propertyID_input"><br>
      <br>
      <input type="submit" value="Delete">
      </form> 
      </body>
      </html>
'''

In [2807]:
@app.route("/Prop_Update", methods=['GET','POST'])
def update_Prop():
   conn= establish_connection()
   cursor = conn.cursor()
   user_id = session.get('UserID')

   if not user_id:
      return redirect('/Login')
   
   try:
      if request.method == 'POST':

         
         role = get_role(user_id,cursor)
         if role != 'agent':
            return "Access denied!", 403
       
         PropertyID = request.form['PropertyID']
         field= request.form['field']
         new_value= request.form['new_value']

         
         cursor.execute("SELECT * FROM Property WHERE propertyID = %s AND agent_UserID = %s::SMALLINT;", (PropertyID, user_id))
         if not cursor.fetchone():
            return redirect('/Properties')
       
         fields_available = ['type','city','state','description','price','availabiliity_status','agency','agent_UserID']
         fields_address = ['number','street','apt','city','state','zip','country']
         fields_apt=['aptroom_num','aptsq_ft','apt_type']
         fields_house = ['room_num','hsq_ft','building_type']
         fields_commercial=['CB_sq_ft','CB_type']

         query = ""
         values = ()


         if field in fields_available:
            query = f"UPDATE property SET {field} = %s WHERE propertyID = %s AND agent_UserID = %s;"
            values = (new_value,PropertyID, user_id)
         elif field in fields_address:
            query = f"UPDATE prop_adress SET {field} = %s WHERE propertyID = %s;"
            values = (new_value,PropertyID, user_id)
         elif field in fields_apt:
            cursor.execute("SELECT 1 FROM property_apt WHERE propertyID = %s;",(PropertyID,))
            if cursor.fetchone():
               query = f"UPDATE property_apt SET {field} = %s WHERE propertyID = %s;"
               values = (new_value,PropertyID)
         elif field in fields_house:
            cursor.execute("SELECT 1 FROM property_house WHERE propertyID = %s;",(PropertyID,))
            if cursor.fetchone():
               query = f"UPDATE property_house SET {field} = %s WHERE propertyID = %s;"
               values = (new_value,PropertyID)  
         elif field in fields_commercial:
            cursor.execute("SELECT 1 FROM property_commercial WHERE propertyID = %s;",(PropertyID,))
            if cursor.fetchone():
               query = f"UPDATE property_commercial SET {field} = %s WHERE propertyID = %s;"
               values = (new_value,PropertyID)    
         if query:
            cursor.execute(query,values)
            conn.commit()
            return redirect('/Properties')
         else:
            return "Invalid field or unsupported type for field",400

   except Exception as e:
      return f"CC error: {e}"
   finally:
      if conn is not None:
         conn.close()

   return f'''<!DOCTYPE html>
         <html lang="en">
         <body>
            <h1> Update a Property </h1>
         </body>

         <form method = "POST"> 

            PropertyID:
         <input type="text" name="PropertyID" required><br>
            <label for="field">Field to change:</label>
            <select name = "field" required>
               <option value="type"> Type</option>
               <option value="city"> City</option> 
               <option value="state"> State</option> 
               <option value="description"> Description</option> 
               <option value="price"> Price</option>
               <option value="availability_status"> Status </option> 
               <option value="agency"> Agency</option> 
               <option value="agent_UserID"> Agent's ID</option> 
               <option value="aptroom_num"> Room Number for Apt</option>
               <option value="aptsq_ft"> SQ FT APT</option>
               <option value="apt_type"> APT Type</option>
               <option value="room_num"> Home Room Number</option>
               <option value="hsq_ft"> SQ FT House</option>
               <option value="building_type"> Building Type</option>
               <option value="CB_sq_ft"> SQ FT Commercial</option>
               <option value="CB_type"> Builidng Type(CB) </option>
               <option value="number"> Home Number</option>
               <option value="street"> Street</option>
               <option value="apt"> APT Number</option>
               <option value="zip"> ZIP Code</option>
               <option value="country"> Country</option>
            </select>
            <br>
            New Value:
         <input type="text" name="new_value"><br>
         <a href = "/dashboard/{user_id}" style = "display: inline-block; padding:10px 15px; background-color:#d4cfd6; color = black; border-radius: 5px; target="_self"> Back to Dashboard </a>


         <input type="submit" value="Update">
         </form> 
         </body>
         </html>
         '''




In [2808]:
@app.route("/BookProp", methods=['GET','POST'])
def bookProperties():
   conn= establish_connection()
   cursor = conn.cursor()
   user_id = session.get('UserID')

   if not user_id:
            return redirect('/Login')
   
   try:
      if request.method == 'POST':
          
         role = get_role(user_id,cursor)
         if role != 'renter':
            return "Access denied!", 403
         
         PropertyID= request.form['PropertyID']
         CardID = request.form['CardID']
         Book_date = request.form['Book_date']
         MoveIn_date=request.form['MoveIn_date']

         cursor.execute("SELECT price FROM property WHERE PropertyID = %s;",(PropertyID,))
         Price = cursor.fetchone()[0]

         cursor.execute("INSERT INTO Booking(UserID,propertyID,CardID,Book_date, MoveIn_date,Price) VALUES (%s::SMALLINT,%s,%s,%s,%s,%s);",(user_id,PropertyID,CardID,Book_date,MoveIn_date,Price))
         conn.commit() 
         return redirect(f'/dashboard/{user_id}')
    
   except Exception as e:
      return f"CC error: {e}"
   finally:
      if conn is not None:
         conn.close()
   
   return '''   <!DOCTYPE html>
      <html lang="en">
      <body>
      <h1> Book a Property </h1>
      </body>

      <form method = "POST"> 
      PropertyID:
      <input type="text" name="PropertyID"><br>
      Card ID:
       <input type="text" name="CardID"><br>
      Book_date:
      <input type="text" name="Book_date"><br>
      MoveIn_date:
      <input type="text" name="MoveIn_date"><br>

      <input type="submit" value="Book">
      </form> 
      </body>
      </html>


      '''

In [2809]:
@app.route("/Bookings", methods=['GET','POST'])
def viewBookings():
   conn= establish_connection()
   cursor = conn.cursor()
   user_id = session.get('UserID')

   if not user_id:
            return redirect('/Login')
   
   try:
          
         role = get_role(user_id,cursor)
         if role != 'renter':
             return redirect('/Bookings_Agent')
             

         cursor.execute('SELECT b.bookingID, b.Book_date, b.MoveIn_date, b.Price, p."type", p.description, a.city, a.state FROM property p JOIN Prop_Adress a On p.propertyID =a.propertyID JOIN Booking b ON p.propertyID = b.propertyID WHERE b.userid = %s;',(user_id,))
         bookings = cursor.fetchall()
             
         rows = ""
         for booking in bookings:
            row = "<tr>" + "".join(f"<td>{value}</td>" for value in booking) + "</tr>\n"
            rows+= row
    
              
   except Exception as e:
      return f"CC error: {e}"
   finally:
      if conn is not None:
         conn.close()

   return f'''
    <!DOCTYPE html>
<html lang="en">
 
<head>
    <meta charset="UTF-8">
    <title> My Bookings </title>

    <style>
        table {{margin: 0 auto, font-size: medium; border: 1px solid black;}}

        h1 {{text-align: center; color =black; font-size: large; font-family: 'Calibri' }}
        td {{background-color:#E4F5D4; border: 1px solid black;}}
        th,
        td{{font-weight: bold; border: 1px solid black; padding: 10px; text-align: center;}}

        td {{font-weight: lighter;}}
    </style>
</head>
<body>
    <h1> Your Bookings <h1>
    <form method = "GET">
                <a href = "/dashboard/{user_id}" style = "display: inline-block; padding:10px 15px; background-color:#d4cfd6; color = black; border-radius: 5px; target="_self"> Back to Dashboard </a>
                <a href = "/Delete_Booking" style = "display: inline-block; padding:10px 15px; background-color:#d4cfd6; color = black; border-radius: 5px; target="_self"> Delete Booking </a>

    <section>
    <table>
        <tr>
            <th>BookingID</th>
            <th>Booking Date</th>
            <th>Move In Date</th>
            <th>Price</th>
            <th>Type of Building</th>
            <th>Description</th>
            <th>City</th>
            <th>State</th>
           
        </tr>
        {rows}
        </table>
        </section>
    </body>
    </html>

'''


In [2810]:
@app.route("/Bookings_Agent", methods=['GET','POST'])
def bookProperties_agent():
   conn= establish_connection()
   cursor = conn.cursor()
   user_id = session.get('UserID')

   if not user_id:
            return redirect('/Login')
   
   try:
          
         role = get_role(user_id,cursor)
         if role !='agent':
            return "Access denied!", 403
      
   
         cursor.execute("SELECT agency FROM User_agent WHERE UserID = %s::SMALLINT;", (user_id,))
         agency = cursor.fetchone()[0]
         print('USer:',user_id)
         print('agency',agency)


         cursor.execute('SELECT p.propertyID,b.bookingID, a."number", a.street, a.apt,p.city, p.state,a.zip,a.country,b.Book_date,b.MoveIn_date,b.price,r.first_name,r.last_name,cc.CC_num FROM property p JOIN Prop_Adress a ON p.propertyID=a.propertyID JOIN Booking b ON p.propertyID=b.propertyID JOIN "user" r on b.UserID = r.UserID JOIN CC cc ON b.CardID=cc.CardID WHERE p.agency= %s;', (agency,))
         bookings = cursor.fetchall()
         print('Bookings',bookings)
         rows = ""
         for booking in bookings:
               row = "<tr>" + "".join(f"<td>{value}</td>" for value in booking) + "</tr>\n"
               rows+= row

   except Exception as e:
        print("An ERROR has occurred:", e)
   finally:
        if conn is not None:
            conn.close()
            cursor.close()
            print('Database connection closed.') 

   return f'''
    <!DOCTYPE html>
<html lang="en">
 
<head>
    <meta charset="UTF-8">
    <title> My Bookings </title>

    <style>
        table {{margin: 0 auto, font-size: medium; border: 1px solid black;}}

        h1 {{text-align: center; color =black; font-size: large; font-family: 'Calibri' }}
        td {{background-color:#E4F5D4; border: 1px solid black;}}
        th,
        td{{font-weight: bold; border: 1px solid black; padding: 10px; text-align: center;}}

        td {{font-weight: lighter;}}
    </style>
</head>
<body>
    <h1> Your Bookings </h1>
    <form method = "GET">
                <a href = "/dashboard/{user_id}" style = "display: inline-block; padding:10px 15px; background-color:#d4cfd6; color = black; border-radius: 5px; target="_self"> Back to Dashboard </a>
                <a href = "/Delete_Booking" style = "display: inline-block; padding:10px 15px; background-color:#d4cfd6; color = black; border-radius: 5px; target="_self"> Delete Booking </a>

    <section>
    <table>
        <tr>
            <th>PropertyID</th>
            <th>BookingID</th>
            <th>House Number</th>
            <th>Street</th>
            <th>APT</th>
            <th>City</th>
            <th>State</th>
            <th>ZIP Code</th>
            <th>Country</th>
            <th>Booking Date</th>
            <th>Move In Date</th>
            <th>Price</th>
            <th>First Name</th>
            <th>Last Name</th>
            <th>CC Number Used</th>
        </tr>
        {rows}
        </table>
        </section>
    </body>
    </html>

'''

In [2811]:
@app.route("/Delete_Booking", methods=['GET','POST'])
def booking_delete():
   conn= establish_connection()
   cursor = conn.cursor()
   user_id = session.get('UserID')

   if not user_id:
            return redirect('/Login')
   try:
        if request.method == 'POST':
             
             BookingID = request.form['BookingID']

             cursor.execute('DELETE FROM Booking WHERE BookingID = %s;',(BookingID,))
             conn.commit() 
             return redirect('/Bookings')
        
   except Exception as e:
        print("An ERROR has occurred:", e)
   finally:
        if conn is not None:
            conn.close()
            cursor.close()
            print('Database connection closed.') 

        return f''' <!DOCTYPE html>
<html lang="en">
<body>
    <h1> Delete Booking </h1>
</body>

<form method = "POST"> 
     <a href = "/dashboard/{user_id}" style = "display: inline-block; padding:10px 15px; background-color:#d4cfd6; color = black; border-radius: 5px; target="_self"> Back to Dashboard </a><br>
     

   Booking Number You Want To Delete:
  <input type="text" name="BookingID"><br>

  <input type="submit" value="Delete">
</form> 
</body>
</html>
'''



In [2812]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [10/May/2025 22:19:51] "GET / HTTP/1.1" 302 -
127.0.0.1 - - [10/May/2025 22:19:51] "GET /dashboard/9 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2025 22:19:51] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [10/May/2025 22:19:56] "GET /Login HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2025 22:20:19] "POST /Login HTTP/1.1" 302 -
127.0.0.1 - - [10/May/2025 22:20:19] "GET /register HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2025 22:20:49] "POST /register HTTP/1.1" 302 -
127.0.0.1 - - [10/May/2025 22:20:49] "GET /Address HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2025 22:21:12] "POST /Address HTTP/1.1" 302 -
127.0.0.1 - - [10/May/2025 22:21:12] "GET / HTTP/1.1" 302 -
127.0.0.1 - - [10/May/2025 22:21:12] "GET /dashboard/10 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2025 22:21:12] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [10/May/2025 22:21:20] "GET /Address_Update HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2025 22:21:30] "POST /Address_Update HT

UserID: 10


127.0.0.1 - - [10/May/2025 22:22:22] "POST /CC_Add HTTP/1.1" 302 -
127.0.0.1 - - [10/May/2025 22:22:22] "GET /CC HTTP/1.1" 200 -


UserID: 10


...

127.0.0.1 - - [10/May/2025 22:22:46] "GET /CC HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2025 22:22:49] "GET /CC_Delete HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2025 22:22:57] "GET /dashboard/10 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2025 22:23:01] "GET /Properties HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2025 22:23:01] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [10/May/2025 22:23:15] "POST /Properties HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2025 22:23:23] "GET /Properties HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2025 22:23:28] "POST /Properties HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2025 22:23:46] "GET /dashboard/10 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2025 22:23:50] "GET /BookProp HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2025 22:23:56] "GET /CC HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2025 22:23:57] "GET /dashboard/10 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2025 22:23:58] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [10/May/2025 22:23:59] "GET /BookProp HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2025 22:2

Database connection closed.


127.0.0.1 - - [10/May/2025 22:24:37] "GET /dashboard/10 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2025 22:24:37] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [10/May/2025 22:24:43] "GET /register HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2025 22:25:10] "POST /register HTTP/1.1" 302 -
127.0.0.1 - - [10/May/2025 22:25:11] "GET /Address HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2025 22:25:28] "POST /Address HTTP/1.1" 302 -
127.0.0.1 - - [10/May/2025 22:25:28] "GET / HTTP/1.1" 302 -
127.0.0.1 - - [10/May/2025 22:25:28] "GET /dashboard/11 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2025 22:25:35] "GET /CC HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2025 22:25:35] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [10/May/2025 22:25:43] "GET /Properties HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2025 22:25:58] "GET /Delete_Properties HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2025 22:26:03] "GET /Prop_Update HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2025 22:26:11] "GET /dashboard/11 HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2025 22:

USer: 11
agency Flowers
Bookings []
Database connection closed.


127.0.0.1 - - [10/May/2025 22:26:21] "GET /dashboard/11 HTTP/1.1" 200 -
